In [1]:
# For initiating and playing the game
import gym
# for creating dictionary and counter
import collections
# for keeping logs
from tensorboardX import SummaryWriter

In [2]:
# Game environment
ENV_NAME = "FrozenLake-v0"
# Gamma variable used as discount factor
GAMMA = 0.9
# Learning Rate
ALPHA = 0.2
# No. of test episodes
TEST_EPISODES = 20

In [6]:
class Agent:
    '''
    Creating a class for agent that will play in the given environment
    '''
    def __init__(self):
        self.env = gym.make(ENV_NAME)
        self.state = self.env.reset()
        self.values = collections.defaultdict(float)
        
    def sample_env(self):
        # Returning sample of state, action and reward
        action = self.env.action_space.sample()
        old_state = self.state
        new_state, reward, is_done, _ = self.env.step(action)
        self.state = self.env.reset() if is_done else new_state
        return old_state, action, reward, new_state
    
    def best_value_and_action(self, state):
        # Selecting the best action for given state
        best_action, best_value = None, None
        for action in range(self.env.action_space.n):
            action_value = self.values[(state, action)]
            if best_value is None or best_value < action_value:
                best_value = action_value
                best_action = action
        return best_value, best_action
    
    def value_update(self, s, a, r, next_s):
        # Calculating the action value, Q, and updating the value, V.
        best_v, _ = self.best_value_and_action(next_s)
        new_v = r + GAMMA * best_v
        old_v = self.values[(s, a)]
        self.values[(s, a)] = old_v * (1 - ALPHA) + new_v * ALPHA
    
    def play_episode(self, env):
        # Playing the game with the best actions for each state
        total_reward = 0.0
        state = env.reset()
        while True:
            _, action = self.best_value_and_action(state)
            new_state, reward, is_done, _ = env.step(action)
            total_reward += reward
            if is_done:
                break
            state = new_state
        return total_reward

In [7]:
test_env = gym.make(ENV_NAME)
agent = Agent()
writer = SummaryWriter(comment='-q-learning')

iter_no = 0
best_reward = 0.0
while True:
    iter_no += 1
    # Playing random game 100 times
    s, a, r, next_s = agent.sample_env()
    # Updating the values
    agent.value_update(s, a, r, next_s)
    
    reward = 0.0
    for _ in range(TEST_EPISODES):
        reward += agent.play_episode(test_env)
    # Dividing the reward by no. of total episodes
    reward /= TEST_EPISODES
    writer.add_scalar("reward", reward, iter_no)
    # Printing only the best reward
    if reward > best_reward:
        print("Best reward updated %.3f -> %.3f" % (
            best_reward, reward))
        best_reward = reward
    if reward > 0.80:
        print("Solved in %d iterations!" % iter_no)
        break
writer.close()

Best reward updated 0.000 -> 0.050
Best reward updated 0.050 -> 0.200
Best reward updated 0.200 -> 0.250
Best reward updated 0.250 -> 0.300
Best reward updated 0.300 -> 0.400
Best reward updated 0.400 -> 0.500
Best reward updated 0.500 -> 0.550
Best reward updated 0.550 -> 0.600
Best reward updated 0.600 -> 0.650
Best reward updated 0.650 -> 0.700
Best reward updated 0.700 -> 0.750
Best reward updated 0.750 -> 0.800
Best reward updated 0.800 -> 0.850
Solved in 3698 iterations!
